In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
import matplotlib.pylab as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.metrics import make_scorer
from sklearn.cross_validation import StratifiedShuffleSplit
from matplotlib.colors import LogNorm
from sklearn.decomposition import PCA
from keras.layers.advanced_activations import PReLU
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.utils import np_utils
from copy import deepcopy
# %matplotlib inline  

/Library/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
Using Theano backend.


In [2]:
trainDF=pd.read_csv("../data/train.csv")

In [3]:
xy_scaler=preprocessing.StandardScaler()
print "abc"
xy_scaler.fit(trainDF[["X","Y"]])
print "abc"
trainDF[["X","Y"]]=xy_scaler.transform(trainDF[["X","Y"]])
print "abc"
trainDF=trainDF[abs(trainDF["Y"])<100]
print "abc"
trainDF.index=range(len(trainDF))
# plt.plot(trainDF["X"],trainDF["Y"],'.')
# plt.show()

abc
abc
abc
abc


In [4]:
NX=100
NY=100
groups = trainDF.groupby('Category')
ii=1
plt.figure(figsize=(20, 20))
for name, group in groups:
    plt.subplot(8,5,ii)
    histo, xedges, yedges = np.histogram2d(np.array(group.X),np.array(group.Y), bins=(NX,NY))
    myextent  =[xedges[0],xedges[-1],yedges[0],yedges[-1]]
    plt.imshow(histo.T,origin='low',extent=myextent,interpolation='nearest',aspect='auto',norm=LogNorm())
    plt.title(name)
#     plt.figure(ii)
#     plt.plot(group.X,group.Y,'.')
    ii+=1
del groups

In [5]:
def parse_time(x):
    DD=datetime.strptime(x,"%Y-%m-%d %H:%M:%S")
    time=DD.hour#*60+DD.minute
    day=DD.day
    month=DD.month
    year=DD.year
    return time,day,month,year

def get_season(x):
    summer=0
    fall=0
    winter=0
    spring=0
    if (x in [5, 6, 7]):
        summer=1
    if (x in [8, 9, 10]):
        fall=1
    if (x in [11, 0, 1]):
        winter=1
    if (x in [2, 3, 4]):
        spring=1
    return summer, fall, winter, spring

In [6]:
def parse_data(df,logodds,logoddsPA):
    feature_list=df.columns.tolist()
    if "Descript" in feature_list:
        feature_list.remove("Descript")
    if "Resolution" in feature_list:
        feature_list.remove("Resolution")
    if "Category" in feature_list:
        feature_list.remove("Category")
    if "Id" in feature_list:
        feature_list.remove("Id")
    cleanData=df[feature_list]
    cleanData.index=range(len(df))
    print "Creating address features"
    address_features=cleanData["Address"].apply(lambda x: logodds[x])
    address_features.columns=["logodds"+str(x) for x in range(len(address_features.columns))]
    print "Parsing dates"
    cleanData["Time"], cleanData["Day"], cleanData["Month"], cleanData["Year"]=zip(*cleanData["Dates"].apply(parse_time))
#     dummy_ranks_DAY = pd.get_dummies(cleanData['DayOfWeek'], prefix='DAY')
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
#     cleanData["DayOfWeek"]=cleanData["DayOfWeek"].apply(lambda x: days.index(x)/float(len(days)))
    print "Creating one-hot variables"
    dummy_ranks_PD = pd.get_dummies(cleanData['PdDistrict'], prefix='PD')
    dummy_ranks_DAY = pd.get_dummies(cleanData["DayOfWeek"], prefix='DAY')
    cleanData["IsInterection"]=cleanData["Address"].apply(lambda x: 1 if "/" in x else 0)
    cleanData["logoddsPA"]=cleanData["Address"].apply(lambda x: logoddsPA[x])
    print "droping processed columns"
    cleanData=cleanData.drop("PdDistrict",axis=1)
    cleanData=cleanData.drop("DayOfWeek",axis=1)
    cleanData=cleanData.drop("Address",axis=1)
    cleanData=cleanData.drop("Dates",axis=1)
    feature_list=cleanData.columns.tolist()
    print "joining one-hot features"
    features = cleanData[feature_list].join(dummy_ranks_PD.ix[:,:]).join(dummy_ranks_DAY.ix[:,:]).join(address_features.ix[:,:])
    print "creating new features"
    features["IsDup"]=pd.Series(features.duplicated()|features.duplicated(take_last=True)).apply(int)
    features["Awake"]=features["Time"].apply(lambda x: 1 if (x==0 or (x>=8 and x<=23)) else 0)
    features["Summer"], features["Fall"], features["Winter"], features["Spring"]=zip(*features["Month"].apply(get_season))
    if "Category" in df.columns:
        print df["Category"]
        labels = df["Category"].astype('category')
        #labels = df["Category"].unique()
#         label_names=labels.unique()
#         labels=labels.cat.rename_categories(range(len(label_names)))
    else:
        labels=None
    return features,labels

In [7]:
addresses=sorted(trainDF["Address"].unique())
categories=sorted(trainDF["Category"].unique())
C_counts=trainDF.groupby(["Category"]).size()
A_C_counts=trainDF.groupby(["Address","Category"]).size()
A_counts=trainDF.groupby(["Address"]).size()
logodds={}
logoddsPA={}
MIN_CAT_COUNTS=2
default_logodds=np.log(C_counts/len(trainDF))-np.log(1.0-C_counts/float(len(trainDF)))
for addr in addresses:
    PA=A_counts[addr]/float(len(trainDF))
    logoddsPA[addr]=np.log(PA)-np.log(1.-PA)
    logodds[addr]=deepcopy(default_logodds)
    for cat in A_C_counts[addr].keys():
        if (A_C_counts[addr][cat]>MIN_CAT_COUNTS) and A_C_counts[addr][cat]<A_counts[addr]:
            PA=A_C_counts[addr][cat]/float(A_counts[addr])
            logodds[addr][categories.index(cat)]=np.log(PA)-np.log(1.0-PA)
    logodds[addr]=pd.Series(logodds[addr])
    logodds[addr].index=range(len(categories))

In [8]:
features, labels=parse_data(trainDF,logodds,logoddsPA)

Creating address features
Parsing dates


/Library/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Creating one-hot variables
droping processed columns
joining one-hot features
creating new features


/Library/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:35: FutureWarning: the take_last=True keyword is deprecated, use keep='last' instead


0                       WARRANTS
1                 OTHER OFFENSES
2                 OTHER OFFENSES
3                  LARCENY/THEFT
4                  LARCENY/THEFT
5                  LARCENY/THEFT
6                  VEHICLE THEFT
7                  VEHICLE THEFT
8                  LARCENY/THEFT
9                  LARCENY/THEFT
10                 LARCENY/THEFT
11                OTHER OFFENSES
12                     VANDALISM
13                 LARCENY/THEFT
14                  NON-CRIMINAL
15                  NON-CRIMINAL
16                       ROBBERY
17                       ASSAULT
18                OTHER OFFENSES
19                  NON-CRIMINAL
20                 LARCENY/THEFT
21                       ROBBERY
22                      WARRANTS
23                  NON-CRIMINAL
24                 LARCENY/THEFT
25                  NON-CRIMINAL
26                 LARCENY/THEFT
27                 LARCENY/THEFT
28                 LARCENY/THEFT
29                OTHER OFFENSES
          

In [9]:
print features.columns.tolist()
print len(features.columns)

['X', 'Y', 'Time', 'Day', 'Month', 'Year', 'IsInterection', 'logoddsPA', 'PD_BAYVIEW', 'PD_CENTRAL', 'PD_INGLESIDE', 'PD_MISSION', 'PD_NORTHERN', 'PD_PARK', 'PD_RICHMOND', 'PD_SOUTHERN', 'PD_TARAVAL', 'PD_TENDERLOIN', 'DAY_Friday', 'DAY_Monday', 'DAY_Saturday', 'DAY_Sunday', 'DAY_Thursday', 'DAY_Tuesday', 'DAY_Wednesday', 'logodds0', 'logodds1', 'logodds2', 'logodds3', 'logodds4', 'logodds5', 'logodds6', 'logodds7', 'logodds8', 'logodds9', 'logodds10', 'logodds11', 'logodds12', 'logodds13', 'logodds14', 'logodds15', 'logodds16', 'logodds17', 'logodds18', 'logodds19', 'logodds20', 'logodds21', 'logodds22', 'logodds23', 'logodds24', 'logodds25', 'logodds26', 'logodds27', 'logodds28', 'logodds29', 'logodds30', 'logodds31', 'logodds32', 'logodds33', 'logodds34', 'logodds35', 'logodds36', 'logodds37', 'logodds38', 'IsDup', 'Awake', 'Summer', 'Fall', 'Winter', 'Spring']
70


In [10]:
# num_feature_list=["Time","Day","Month","Year","DayOfWeek"]
collist=features.columns.tolist()
scaler = preprocessing.StandardScaler()
scaler.fit(features)
features[collist]=scaler.transform(features)

In [11]:
print features[1:10]

          X         Y      Time       Day     Month      Year  IsInterection  \
1 -0.123732  0.313018  1.463812 -0.292669 -0.418903  1.731591       1.538547   
2 -0.063274  1.381346  1.463812 -0.292669 -0.418903  1.731591       1.538547   
3 -0.167381  1.400312  1.463812 -0.292669 -0.418903  1.731591      -0.649964   
4 -0.631787  0.186493  1.463812 -0.292669 -0.418903  1.731591      -0.649964   
5  0.771649 -2.218282  1.463812 -0.292669 -0.418903  1.731591      -0.649964   
6 -0.022298 -1.733799  1.463812 -0.292669 -0.418903  1.731591       1.538547   
7  2.036376 -1.633403  1.463812 -0.292669 -0.418903  1.731591       1.538547   
8 -3.378776  0.395894  1.463812 -0.292669 -0.418903  1.731591      -0.649964   
9  0.145365  1.687050  1.463812 -0.292669 -0.418903  1.731591       1.538547   

   logoddsPA  PD_BAYVIEW  PD_CENTRAL    ...     logodds35  logodds36  \
1  -0.645441   -0.336748   -0.328369    ...      0.172386   0.668083   
2  -1.009236   -0.336748   -0.328369    ...      0.1723

In [12]:
new_PCA=PCA(n_components=60)
new_PCA.fit(features)
plt.plot(new_PCA.explained_variance_ratio_)
plt.yscale('log')
plt.title("PCA explained ratio of features")
print new_PCA.explained_variance_ratio_

[ 0.07714363  0.06267647  0.04861061  0.04300871  0.03340587  0.03085896
  0.02834616  0.02731142  0.02509795  0.02393235  0.02272111  0.02029053
  0.02000664  0.019452    0.01918369  0.01820193  0.01729548  0.01724542
  0.01715838  0.01714426  0.01703779  0.01695665  0.01668107  0.01651773
  0.01626539  0.01588035  0.01489293  0.01472615  0.01422875  0.01412131
  0.0136947   0.01330571  0.01234474  0.01181851  0.01158515  0.01126484
  0.01104143  0.0105683   0.01032344  0.00943963  0.00903894  0.00838257
  0.00819149  0.00795652  0.0074206   0.00726109  0.00707183  0.00675531
  0.0067189   0.00646287  0.00634686  0.00612685  0.0060737   0.00556134
  0.00526841  0.00516993  0.0048638   0.00452517  0.00417899  0.00407036]


In [13]:
plt.plot(new_PCA.explained_variance_ratio_.cumsum())
plt.title("cumsum of PCA explained ratio")

In [13]:
sss = StratifiedShuffleSplit(labels, train_size=0.75)
for train_index, test_index in sss:
    features_train,features_test=features.iloc[train_index],features.iloc[test_index]
    labels_train,labels_test=labels[train_index],labels[test_index]
features_test.index=range(len(features_test))
features_train.index=range(len(features_train))
labels_train.index=range(len(labels_train))
labels_test.index=range(len(labels_test))
features.index=range(len(features))
labels.index=range(len(labels))

In [14]:
def build_and_fit_model(X_train,y_train,X_test=None,y_test=None,hn=32,dp=0.5,layers=1,epochs=1,batches=64,verbose=0):
    input_dim=X_train.shape[1]
    output_dim=len(y_train.unique())
    Y_train=np_utils.to_categorical(y_train.cat.rename_categories(range(len(y_train.unique()))))
    print output_dim, input_dim
    model = Sequential()
    model.add(Dense(hn, input_shape=(input_dim,)))
    model.add(PReLU())
    model.add(Dropout(dp))

    for i in range(layers):
        model.add(Dense(hn))
        model.add(PReLU())
        model.add(BatchNormalization())
        model.add(Dropout(dp))

    model.add(Dense(output_dim))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    if X_test is not None:
        Y_test=np_utils.to_categorical(y_test.cat.rename_categories(range(len(y_test.unique()))))
        fitting=model.fit(X_train, Y_train, nb_epoch=epochs, batch_size=batches,verbose=verbose,validation_data=(X_test,Y_test))
        test_score = log_loss(y_test, model.predict_proba(X_test,verbose=0))
    else:
        model.fit(X_train, Y_train, nb_epoch=epochs, batch_size=batches,verbose=verbose)
        fitting=0
        test_score = 0
    return test_score, fitting, model

In [15]:
len(features.columns)

70

In [16]:
N_EPOCHS=20
N_HN=128
N_LAYERS=1
DP=0.5

In [17]:
score, fitting, model = build_and_fit_model(features_train.as_matrix(),labels_train,X_test=features_test.as_matrix(),y_test=labels_test,hn=N_HN,layers=N_LAYERS,epochs=N_EPOCHS,verbose=2,dp=DP)

39 70
Train on 658486 samples, validate on 219496 samples
Epoch 1/20
39s - loss: 2.3799 - val_loss: 2.2118
Epoch 2/20
38s - loss: 2.2775 - val_loss: 2.2022
Epoch 3/20
42s - loss: 2.2664 - val_loss: 2.1973
Epoch 4/20
43s - loss: 2.2604 - val_loss: 2.1911
Epoch 5/20
40s - loss: 2.2554 - val_loss: 2.1907
Epoch 6/20
40s - loss: 2.2528 - val_loss: 2.1871
Epoch 7/20
41s - loss: 2.2507 - val_loss: 2.1859
Epoch 8/20
43s - loss: 2.2488 - val_loss: 2.1864
Epoch 9/20
41s - loss: 2.2478 - val_loss: 2.1825
Epoch 10/20
44s - loss: 2.2472 - val_loss: 2.1846
Epoch 11/20
43s - loss: 2.2458 - val_loss: 2.1822
Epoch 12/20
41s - loss: 2.2444 - val_loss: 2.1822
Epoch 13/20
42s - loss: 2.2439 - val_loss: 2.1824
Epoch 14/20
42s - loss: 2.2435 - val_loss: 2.1789
Epoch 15/20
42s - loss: 2.2432 - val_loss: 2.1821
Epoch 16/20
43s - loss: 2.2430 - val_loss: 2.1796
Epoch 17/20
42s - loss: 2.2426 - val_loss: 2.1783
Epoch 18/20
42s - loss: 2.2413 - val_loss: 2.1780
Epoch 19/20
43s - loss: 2.2411 - val_loss: 2.1782
E

In [18]:
print "all", log_loss(labels, model.predict_proba(features.as_matrix(),verbose=0))
print "train", log_loss(labels_train, model.predict_proba(features_train.as_matrix(),verbose=0))
print "test", log_loss(labels_test, model.predict_proba(features_test.as_matrix(),verbose=0))

all 2.17621264277
train 2.17585184371
test 2.17729503668


In [19]:

plt.plot(fitting.history['val_loss'],label="validation")
plt.plot(fitting.history['loss'],label="train")
# plt.xscale('log')
plt.legend()

In [19]:
score, fitting, model = build_and_fit_model(features.as_matrix(),labels,hn=N_HN,layers=N_LAYERS,epochs=N_EPOCHS,verbose=2,dp=DP)

39 70
Epoch 1/20
51s - loss: 2.3543
Epoch 2/20
54s - loss: 2.2705
Epoch 3/20
52s - loss: 2.2614
Epoch 4/20
51s - loss: 2.2553
Epoch 5/20
51s - loss: 2.2516
Epoch 6/20
51s - loss: 2.2502
Epoch 7/20
51s - loss: 2.2476
Epoch 8/20
51s - loss: 2.2469
Epoch 9/20
51s - loss: 2.2445
Epoch 10/20
53s - loss: 2.2436
Epoch 11/20
52s - loss: 2.2429
Epoch 12/20
52s - loss: 2.2421
Epoch 13/20
51s - loss: 2.2421
Epoch 14/20
52s - loss: 2.2409
Epoch 15/20
53s - loss: 2.2402
Epoch 16/20
52s - loss: 2.2403
Epoch 17/20
51s - loss: 2.2396
Epoch 18/20
51s - loss: 2.2392
Epoch 19/20
52s - loss: 2.2385
Epoch 20/20
51s - loss: 2.2383


In [20]:
print "all", log_loss(labels, model.predict_proba(features.as_matrix(),verbose=0))
print "train", log_loss(labels_train, model.predict_proba(features_train.as_matrix(),verbose=0))
print "test", log_loss(labels_test, model.predict_proba(features_test.as_matrix(),verbose=0))

all 2.17052233917
train 2.1714598503
test 2.1677098143


In [21]:
testDF=pd.read_csv("../data/test.csv")
testDF[["X","Y"]]=xy_scaler.transform(testDF[["X","Y"]])
#set outliers to 0
testDF["X"]=testDF["X"].apply(lambda x: 0 if abs(x)>5 else x)
testDF["Y"]=testDF["Y"].apply(lambda y: 0 if abs(y)>5 else y)

In [22]:
new_addresses=sorted(testDF["Address"].unique())
new_A_counts=testDF.groupby("Address").size()
only_new=set(new_addresses+addresses)-set(addresses)
only_old=set(new_addresses+addresses)-set(new_addresses)
in_both=set(new_addresses).intersection(addresses)
for addr in only_new:
    PA=new_A_counts[addr]/float(len(testDF)+len(trainDF))
    logoddsPA[addr]=np.log(PA)-np.log(1.-PA)
    logodds[addr]=deepcopy(default_logodds)
    logodds[addr].index=range(len(categories))
for addr in in_both:
    PA=(A_counts[addr]+new_A_counts[addr])/float(len(testDF)+len(trainDF))
    logoddsPA[addr]=np.log(PA)-np.log(1.-PA)

In [23]:
features_sub, _=parse_data(testDF,logodds,logoddsPA)
# scaler.fit(features_test)

Creating address features
Parsing dates
Creating one-hot variables
droping processed columns
joining one-hot features
creating new features


/Library/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:35: FutureWarning: the take_last=True keyword is deprecated, use keep='last' instead


In [24]:
collist=features_sub.columns.tolist()
print collist

['X', 'Y', 'Time', 'Day', 'Month', 'Year', 'IsInterection', 'logoddsPA', 'PD_BAYVIEW', 'PD_CENTRAL', 'PD_INGLESIDE', 'PD_MISSION', 'PD_NORTHERN', 'PD_PARK', 'PD_RICHMOND', 'PD_SOUTHERN', 'PD_TARAVAL', 'PD_TENDERLOIN', 'DAY_Friday', 'DAY_Monday', 'DAY_Saturday', 'DAY_Sunday', 'DAY_Thursday', 'DAY_Tuesday', 'DAY_Wednesday', 'logodds0', 'logodds1', 'logodds2', 'logodds3', 'logodds4', 'logodds5', 'logodds6', 'logodds7', 'logodds8', 'logodds9', 'logodds10', 'logodds11', 'logodds12', 'logodds13', 'logodds14', 'logodds15', 'logodds16', 'logodds17', 'logodds18', 'logodds19', 'logodds20', 'logodds21', 'logodds22', 'logodds23', 'logodds24', 'logodds25', 'logodds26', 'logodds27', 'logodds28', 'logodds29', 'logodds30', 'logodds31', 'logodds32', 'logodds33', 'logodds34', 'logodds35', 'logodds36', 'logodds37', 'logodds38', 'IsDup', 'Awake', 'Summer', 'Fall', 'Winter', 'Spring']


In [25]:
features_sub[collist]=scaler.transform(features_sub[collist])

predDF=pd.DataFrame(model.predict_proba(features_sub.as_matrix(),verbose=0),columns=sorted(labels.unique()))
predDF.head()

,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0.007325,0.080857,0.000033,0.000029,0.048726,0.000864,0.001179,0.004372,0.000744,0.000194,...,0.000119,0.000345,0.000638,0.035258,1.674602e-07,0.003791,0.122631,0.135539,0.015837,0.004196
1,0.003505,0.095362,0.000009,0.000032,0.001960,0.000576,0.008536,0.011443,0.001551,0.000031,...,0.000024,0.000490,0.000035,0.031019,1.405286e-08,0.000590,0.052829,0.089445,0.035631,0.010354
2,0.002468,0.017544,0.000030,0.000002,0.155895,0.000273,0.001154,0.000436,0.000073,0.000256,...,0.000010,0.000080,0.000374,0.017053,7.101684e-09,0.001116,0.070508,0.174402,0.002023,0.000632
3,0.001416,0.284944,0.000045,0.001642,0.015782,0.003688,0.006474,0.056990,0.032637,0.000173,...,0.000087,0.012184,0.000550,0.037635,3.686594e-07,0.006069,0.044659,0.007146,0.068627,0.045664
4,0.001416,0.284944,0.000045,0.001642,0.015782,0.003688,0.006474,0.056990,0.032637,0.000173,...,0.000087,0.012184,0.000550,0.037635,3.686594e-07,0.006069,0.044659,0.007146,0.068627,0.045664


In [26]:
predDF.to_csv("Crime_SF_NN_Logodds.csv",index_label="Id",na_rep="0")

In [27]:
#############xgboost implementation#############
import xgboost as xgb


In [28]:
print labels[1:10]

1    OTHER OFFENSES
2    OTHER OFFENSES
3     LARCENY/THEFT
4     LARCENY/THEFT
5     LARCENY/THEFT
6     VEHICLE THEFT
7     VEHICLE THEFT
8     LARCENY/THEFT
9     LARCENY/THEFT
Name: Category, dtype: category
Categories (39, object): [ARSON, ASSAULT, BAD CHECKS, BRIBERY, ..., VANDALISM, VEHICLE THEFT, WARRANTS, WEAPON LAWS]


In [30]:

from sklearn.preprocessing import LabelEncoder

In [35]:
### converting labels
le = LabelEncoder()
le.fit(labels)
list(le.classes_)
labels_num = le.transform(labels)


In [36]:
sss = StratifiedShuffleSplit(labels_num, train_size=0.75)
for train_index, test_index in sss:
    features_train,features_test=features.iloc[train_index],features.iloc[test_index]
    labels_train,labels_test=labels_num[train_index],labels_num[test_index]
features_test.index=range(len(features_test))
features_train.index=range(len(features_train))
labels_train.index=range(len(labels_train))
labels_test.index=range(len(labels_test))
features.index=range(len(features))
labels_num.index=range(len(labels_num))

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [37]:
xg_train = xgb.DMatrix( features_train, label=labels_train)
xg_test = xgb.DMatrix(features_test, label=labels_test)

In [40]:
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softprob'
# scale weight of positive examples
param['eta'] = 0.1
param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 39
param['eval_metric'] = 'mlogloss'

In [ ]:
watchlist = [ (xg_train,'train'), (xg_test, 'test') ]
num_round = 5
bst = xgb.train(param, xg_train, num_round, watchlist );
# get prediction
pred = bst.predict( xg_test );


[0]	train-merror:0.659853	test-merror:0.663083
[1]	train-merror:0.657615	test-merror:0.660554
